<a href="https://colab.research.google.com/github/jimmyQuigeeAnalytics/data-extraction/blob/main/Google_Trends_Related_Queries_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Background

This Jupyter notebook is designed to extract and analyze Google Trends data for various categories and sub-categories of products. The primary goal is to fetch related queries for specific keywords within a two-year timeframe on a global scale.

The process involves the following steps:
1. **Initialization**: The notebook installs and imports necessary libraries, including `pytrends` for interacting with the Google Trends API.
2. **Data Definition**: It defines a comprehensive list of categories and sub-categories, each with a set of relevant product keywords.
3. **Timeframe and Location Setting**: The timeframe is set to the last two years, and the location is set to worldwide.
4. **Data Retrieval**: A function is defined to fetch related queries for each keyword, with built-in error handling and delays to manage rate limits imposed by the Google Trends API.
5. **Data Combination and Storage**: The fetched data is combined into a single dataframe, which is then split into training and reserved datasets.
6. **Saving Results**: The split datasets are saved as CSV files for further analysis.

This notebook automates the process of gathering and organizing Google Trends data, providing a structured approach to analyze trends and related queries for a wide range of product categories.

### Step 1: Install and Import Libraries

In [32]:
! pip install pytrends

In [33]:
from pytrends.request import TrendReq
import datetime
import pandas as pd
from google.colab import files
import time
from pytrends.exceptions import TooManyRequestsError

**Explanation:**
- **Install `pytrends`**: Ensures the library is available for use.
- **Import Libraries**: Imports necessary libraries for Google Trends API requests, date handling, data manipulation, file handling, and time management.

### Step 2: Initialize Pytrends and Define Data

In [34]:
# # Initialize the Google Trends API Client
# pytrend = TrendReq()

# # Confection categories and their respective products
# confections_data = {
#   "Confections": {
#     "Chocolates": ["Milk chocolate", "Dark chocolate", "White chocolate", "Chocolate truffles", "Filled chocolates"],
#     "Hard Candies": ["Lollipops", "Candy canes", "Jolly Ranchers", "Lemon drops"],
#     "Gummies": ["Gummy bears", "Gummy worms", "Sour gummies"],
#     "Jellies": ["Jelly beans", "Pâte de fruits", "Turkish delight"],
#     "Licorice": ["Black licorice", "Red licorice", "Licorice twists"],
#     "Chews": ["Starburst", "Taffy", "Hi-Chew"],
#     "Marshmallows": ["Regular marshmallows", "Mini marshmallows", "Flavored marshmallows"],
#     "Caramels": ["Plain caramels", "Salted caramels", "Chocolate-covered caramels"],
#     "Nougat": ["White nougat (with almonds)", "Brown nougat (with chocolate)"],
#     "Fudge": ["Chocolate fudge", "Peanut butter fudge", "Maple fudge"],
#     "Toffees": ["Butter toffee", "English toffee", "Almond roca"],
#     "Brittles": ["Peanut brittle", "Sesame brittle", "Pecan brittle"],
#     "Truffles": ["Dark chocolate truffles", "Milk chocolate truffles", "White chocolate truffles"],
#     "Pastilles": ["Fruit pastilles", "Licorice pastilles", "Mint pastilles"],
#     "Marzipan": ["Plain marzipan", "Fruit-shaped marzipan", "Marzipan-covered cake"],
#     "Halva": ["Sesame halva", "Pistachio halva", "Almond halva"],
#     "Fondants": ["Plain fondant", "Rose fondant", "Cream fondants"],
#     "Pralines": ["Pecan pralines", "Almond pralines", "Chocolate pralines"],
#     "Mints": ["Peppermint candies", "Spearmint candies", "Breath mints"]
# }
# # (Other category definitions like `alcoholic_data`, `non_alcoholic_data`, etc. follow the same structure)

In [35]:
# Initialize the Google Trends API Client
pytrend = TrendReq()

# Define each category with a list of keywords (products)
confections_data = {
    "Confections": {
    "Chocolates": ["Milk chocolate", "Dark chocolate", "White chocolate", "Chocolate truffles", "Filled chocolates"],
    "Hard Candies": ["Lollipops", "Candy canes", "Jolly Ranchers", "Lemon drops"],
    "Gummies": ["Gummy bears", "Gummy worms", "Sour gummies"],
    "Jellies": ["Jelly beans", "Pâte de fruits", "Turkish delight"],
    "Licorice": ["Black licorice", "Red licorice", "Licorice twists"],
    "Chews": ["Starburst", "Taffy", "Hi-Chew"],
    "Marshmallows": ["Regular marshmallows", "Mini marshmallows", "Flavored marshmallows"],
    "Caramels": ["Plain caramels", "Salted caramels", "Chocolate-covered caramels"],
    "Nougat": ["White nougat (with almonds)", "Brown nougat (with chocolate)"],
    "Fudge": ["Chocolate fudge", "Peanut butter fudge", "Maple fudge"],
    "Toffees": ["Butter toffee", "English toffee", "Almond roca"],
    "Brittles": ["Peanut brittle", "Sesame brittle", "Pecan brittle"],
    "Truffles": ["Dark chocolate truffles", "Milk chocolate truffles", "White chocolate truffles"],
    "Pastilles": ["Fruit pastilles", "Licorice pastilles", "Mint pastilles"],
    "Marzipan": ["Plain marzipan", "Fruit-shaped marzipan", "Marzipan-covered cake"],
    "Halva": ["Sesame halva", "Pistachio halva", "Almond halva"],
    "Fondants": ["Plain fondant", "Rose fondant", "Cream fondants"],
    "Pralines": ["Pecan pralines", "Almond pralines", "Chocolate pralines"],
    "Mints": ["Peppermint candies", "Spearmint candies", "Breath mints"]
    }
}

alcoholic_data = {
    "Alcoholic Beverages": {
    "Beers": ["Lager", "Ale", "Stout", "IPA (Indian Pale Ale)", "Pilsner"],
    "Red Wines": ["Merlot", "Cabernet Sauvignon", "Pinot Noir"],
    "White Wines": ["Chardonnay", "Sauvignon Blanc"],
    "Rosé": ["Sparkling wine","Champagne", "Prosecco"],
    "Whiskeys": ["Scotch", "Bourbon", "Rye whiskey", "Irish whiskey", "Tennessee whiskey"],
    "Vodka": ["Plain vodka", "Flavored vodka (e.g., lemon, berry, vanilla)"],
    "Rum": ["White rum", "Dark rum", "Spiced rum", "Aged rum"],
    "Tequila": ["Blanco (Silver)", "Reposado", "Añejo", "Extra Añejo"],
    "Brandy": ["Cognac", "Armagnac", "Pisco", "Fruit brandies (e.g., apple, pear)"],
    "Gin": ["London dry gin", "Old Tom gin", "Flavored gin", "Plymouth gin"],
    "Liqueurs": ["Baileys (Irish Cream)", "Amaretto", "Kahlúa (coffee liqueur)", "Limoncello", "Cointreau (orange liqueur)"],
    "Aperitifs": ["Campari", "Aperol", "Vermouth"],
    "Digestifs": ["Grappa", "Jägermeister", "Fernet"]
    }
}

# Combine all categories
categories = {
    **confections_data,
    **alcoholic_data,
    # Add other category dictionaries similarly
}

**Explanation:**
- **Initialize Pytrends**: Creates an instance of the `TrendReq` class to interface with the Google Trends API.
- **Define Data**: Lists confection categories and their respective products. Similar structures are used for other categories like alcoholic beverages, non-alcoholic beverages, groceries, etc.
- **Combine Categories**: Merges all category dictionaries into one `categories` dictionary.

### Step 3: Define the Timeframe and Location

In [38]:
# Set the timeframe for the last two years
end_date = datetime.datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.datetime.now() - datetime.timedelta(days=730)).strftime('%Y-%m-%d')
timeframe = f'{start_date} {end_date}'

# Location (assuming worldwide)
location = 'Worldwide'

**Explanation:**
- **Timeframe Calculation**: Sets the data retrieval period to the last two years.
- **Location**: Assumes a global scope for the data.

### Step 4: Initialize Data Storage and Define Fetch Function

In [39]:
all_related_queries = []

# Function to fetch related queries with error handling and delay
def fetch_related_queries(pytrend, category, sub_category, keywords, timeframe, location):
    try:
        pytrend.build_payload(kw_list=keywords, timeframe=timeframe)
        related_queries = pytrend.related_queries()

        for keyword, queries in related_queries.items():
            df = queries['top']  # 'top' gives the top related queries.

            if df is not None:  # Sometimes there might not be any related queries returned.
                df['Keyword'] = keyword
                df['Category'] = category
                df['Sub-Category'] = sub_category
                df['Date Range'] = timeframe
                df['Location'] = location
                all_related_queries.append(df)
    except TooManyRequestsError:
        print(f"Too many requests. Retrying for category: {category}, sub-category: {sub_category}, and keywords: {keywords}")
        time.sleep(60)  # Delay for 60 seconds before retrying
        fetch_related_queries(pytrend, category, sub_category, keywords, timeframe, location)

**Explanation:**
- **Data Storage**: Initializes an empty list to store related queries.
- **Fetch Function**: Defines a function to fetch related queries, handle errors, and add delays to avoid rate limiting.

### Step 5: Fetch Data

In [40]:
# Fetch related queries for each category and sub-category
for category, sub_categories in categories.items():
    if isinstance(sub_categories, dict):
        for sub_category, keywords in sub_categories.items():
            fetch_related_queries(pytrend, category, sub_category, keywords, timeframe, location)
            time.sleep(10)  # Delay between requests to avoid rate limiting
    else:
        print(f"Error: Expected a dictionary for sub-categories in category '{category}', but got {type(sub_categories)}")

**Explanation:**
- **Fetch Data**: Iterates over each category and sub-category to fetch related queries, adding delays between requests.

### Step 6: Combine Dataframes and Save Results

In [41]:
# Combine all the dataframes
if all_related_queries:
    final_df = pd.concat(all_related_queries, ignore_index=True)
    print(final_df)
else:
    print("No related queries found.")

                             query  value         Keyword  \
0            chocolate milk powder    100  Milk chocolate   
1             chocolate milk shake     40  Milk chocolate   
2            costco chocolate milk     14  Milk chocolate   
3                 kitkat chocolate     13  Milk chocolate   
4    dairy milk chocolate flavours      3  Milk chocolate   
..                             ...    ...             ...   
306        ist jägermeister gesund      8    Jägermeister   
307                   fernet drink    100          Fernet   
308               que es el fernet     88          Fernet   
309                fernet de litro     77          Fernet   
310                 what is fernet     71          Fernet   

                Category Sub-Category             Date Range   Location  
0            Confections   Chocolates  2022-07-01 2024-06-30  Worldwide  
1            Confections   Chocolates  2022-07-01 2024-06-30  Worldwide  
2            Confections   Chocolates  2022-0

In [46]:
final_df.head()

,query,value,Keyword,Category,Sub-Category,Date Range,Location
0,chocolate milk powder,100,Milk chocolate,Confections,Chocolates,2022-07-01 2024-06-30,Worldwide
1,chocolate milk shake,40,Milk chocolate,Confections,Chocolates,2022-07-01 2024-06-30,Worldwide
2,costco chocolate milk,14,Milk chocolate,Confections,Chocolates,2022-07-01 2024-06-30,Worldwide
3,kitkat chocolate,13,Milk chocolate,Confections,Chocolates,2022-07-01 2024-06-30,Worldwide
4,dairy milk chocolate flavours,3,Milk chocolate,Confections,Chocolates,2022-07-01 2024-06-30,Worldwide


In [42]:
# Split the dataset into 75% and 25% using random samples
# train_df = final_df.sample(frac=0.75, random_state=42)
# reserved_df = final_df.drop(train_df.index)

In [43]:
# Save each split to separate CSV files
final_df.to_csv('cat1.csv', index=False)
# train_df.to_csv('train.csv', index=False)
# reserved_df.to_csv('reserved.csv', index=False)
final_df.shape
# train_df.shape
# reserved_df.shape
files.download('cat1.csv')
# files.download('train.csv')
# files.download('reserved.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Explanation:**
- **Combine Data**: Concatenates all the individual dataframes into a single dataframe `final_df`.
- **Split Data**: Randomly splits the data into training (75%) and reserved (25%) sets.
- **Save Results**: Saves the split dataframes to CSV files and initiates downloads.